In [ ]:
import napari
import numpy as np
import os
import dask.array as da
import dask

import zarr
from ome_zarr.io import parse_url
from ome_zarr.reader import Reader

### Load segmentation as pyramid

In [ ]:
### Load segmentation
# segmentation is always good if loaded as pyramid
seg_zarr_path = '/mnt/ampa02_data01/tmurakami/240425_whole_4color_2nd_M037-3pb/segmentation/Gad1/segmented_pyramid.zarr'

seg_zarr = zarr.open(seg_zarr_path, mode='r')

seg_pyramid = []
for i in list(seg_zarr.keys()):
    seg_pyramid.append(da.from_zarr(seg_zarr[i]))

### Use downsampled image

In [ ]:
# set target resolution
resolution = 1
segment_chan = 4
scale=(2.0,1.3,1.3)
data_path = '/mnt/ampa02_data01/tmurakami/240425_whole_4color_2nd_M037-3pb/fused/fused.n5'
_, ext = os.path.splitext(data_path)

if ext == '.n5': # n5 assume bigstitcher (bigdataviewer) format
    # load images according to the input parameters.
    img_zarr = zarr.open(store=zarr.N5Store(data_path), mode='r')
    n5_setups = list(img_zarr.keys())
    res_list = list(img_zarr[n5_setups[segment_chan]]['timepoint0'].keys())

    img_da = da.from_zarr(img_zarr[n5_setups[segment_chan]]['timepoint0'][res_list[resolution]])
    img = img_da.compute()
    
elif ext == '.zarr': # zarr assumes ome-zarr
    # read the image data
    store = parse_url(data_path, mode="r").store
    reader = Reader(parse_url(data_path))
    # nodes may include images, labels etc
    nodes = list(reader())
    # first node will be the image pixel data
    image_node = nodes[0]

    dask_data = image_node.data
    img = dask_data[resolution][segment_chan,...].compute()
    
### visualization using napari
viewer = napari.Viewer()
viewer.add_image(img,contrast_limits=[0,20000], rgb=False, name='image', colormap='gray', blending='additive',scale=(2.0,1.3,1.3))
viewer.add_labels(seg_pyramid[resolution],opacity=0.4,scale=(2.0,1.3,1.3))

### Use pyramid resolution

In [ ]:
data_path = '/mnt/ampa02_data01/tmurakami/240425_whole_4color_2nd_M037-3pb/fused/fused.n5'
_, ext = os.path.splitext(data_path)
scale=(2.0,1.3,1.3)

if ext == '.n5': # n5 assume bigstitcher (bigdataviewer) format
    ### if the original file is n5

    # load images according to the input parameters.
    img_zarr = zarr.open(store=zarr.N5Store(data_path), mode='r')
    n5_setups = list(img_zarr.keys())
    res_list = list(img_zarr[n5_setups[0]]['timepoint0'].keys())

    imgs = []
    for setup in n5_setups:
        pyramid = []
        for res in res_list:
            pyramid.append(da.from_zarr(img_zarr[setup]['timepoint0'][res]))
        imgs.append(pyramid)

    viewer = napari.Viewer()
    for img in imgs:
        viewer.add_image(img, contrast_limits=[0,20000], rgb=False, name='image', colormap='gray', blending='additive',scale=scale)
    
elif ext == '.zarr': # zarr assumes ome-zarr
    viewer = napari.Viewer()
    viewer.open(data_path, plugin="napari-ome-zarr")
    
viewer.add_labels(seg_pyramid,opacity=0.4,scale=scale)